In [ ]:
import pandas as pd
import numpy as np
import kagglehub
import cupy as cp

In [ ]:
from sklearn.metrics import roc_auc_score

import numpy as np
import cupy as cp
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

class WeatherPredictionNetwork:
    def __init__(self, layers, activations, binary_output, seed=None, l2_lambda=0.01):
        if seed is not None:
            cp.random.seed(seed)

        self.num_layers = len(layers)
        self.weights = []
        self.biases = []
        self.activations = activations
        self.l2_lambda = l2_lambda
        self.binary_output = binary_output

        self.weight_error_history = []  # Initialize for weight updates
        self.bias_error_history = []    # Initialize for bias updates
        self.training_mae = []  # Track MAE for training
        self.training_auc = []  # Track AUC for training
        self.testing_mae = []  # Track MAE for testing
        self.testing_auc = []  # Track AUC for testing

        for i in range(self.num_layers - 1):
            limit = cp.sqrt(6 / (layers[i] + layers[i + 1]))
            self.weights.append(cp.random.uniform(-limit, limit, (layers[i], layers[i + 1])).astype(cp.float32))
            self.biases.append(cp.zeros((1, layers[i + 1]), dtype=cp.float32))

    def apply_activation(self, z, activation):
        if activation == "sigmoid":
            return 1 / (1 + cp.exp(-cp.clip(z, -10, 10)))
        elif activation == "relu":
            return cp.maximum(0, z)
        elif activation == "tanh":
            return cp.tanh(z)
        elif activation == "linear":
            return z
        elif activation == "softmax":
            z_stable = z - cp.max(z, axis=1, keepdims=True)
            exps = cp.exp(z_stable)
            return exps / cp.sum(exps, axis=1, keepdims=True)

    def apply_activation_derivative(self, a, activation):
        if activation == "sigmoid":
            return a * (1 - a)
        elif activation == "relu":
            return cp.where(a > 0, 1, 0)
        elif activation == "tanh":
            return 1 - a**2
        elif activation == "linear":
            return 1

    def forward(self, X):
        self.activations_values = [X]
        self.z_values = []

        for i in range(self.num_layers - 2):
            z = cp.dot(self.activations_values[-1], self.weights[i]) + self.biases[i]
            self.z_values.append(z)
            a = self.apply_activation(z, self.activations[i])
            self.activations_values.append(a)

        z_last = cp.dot(self.activations_values[-1], self.weights[-1]) + self.biases[-1]
        self.z_values.append(z_last)
        
        z_reg, z_cls = z_last[:, 0:1], z_last[:, 1:]
        a_reg = self.apply_activation(z_reg, "linear")
        if self.binary_output:
            a_cls = self.apply_activation(z_cls, "sigmoid")
        else:
            a_cls = self.apply_activation(z_cls, "linear")
        
        self.activations_values.append(cp.hstack([a_reg, a_cls]))

        return self.activations_values[-1]

    def backward(self, X, y, output, learning_rate):
        deltas = []

        y_reg, y_cls = y[:, 0:1], y[:, 1:]
        if self.binary_output:
            output_reg, output_cls = output[:, 0:1], cp.clip(output[:, 1:], 1e-9, 1 - 1e-9)
        else:
            output_reg, output_cls = output[:, 0:1], output[:, 1:]

        reg_error = y_reg - output_reg
        reg_delta = reg_error * self.apply_activation_derivative(output_reg, "linear")

        if self.binary_output:
            cls_error = y_cls - output_cls
            cls_delta = cls_error * self.apply_activation_derivative(output_cls, "sigmoid")
        else:
            cls_error = y_cls - output_cls
            cls_delta = cls_error * self.apply_activation_derivative(output_cls, "linear")

        output_delta = cp.hstack([reg_delta, cls_delta])
        deltas.append(output_delta)

        for i in range(self.num_layers - 2, 0, -1):
            z = self.z_values[i - 1]
            delta = cp.dot(deltas[-1], self.weights[i].T) * self.apply_activation_derivative(self.activations_values[i], self.activations[i - 1])
            deltas.append(delta)

        deltas.reverse()

        for i in range(self.num_layers - 1):
            grad_w = cp.dot(self.activations_values[i].T, deltas[i]) + self.l2_lambda * self.weights[i]
            grad_b = cp.sum(deltas[i], axis=0, keepdims=True)

            grad_w = cp.clip(grad_w, -1.0, 1.0)
            grad_b = cp.clip(grad_b, -1.0, 1.0)

            # Append the norms of gradients to history
            self.weight_error_history.append(cp.linalg.norm(grad_w))
            self.bias_error_history.append(cp.linalg.norm(grad_b))

            self.weights[i] += learning_rate * grad_w
            self.biases[i] += learning_rate * grad_b

    def clip_weights(self, clip_value=1.0):
        for i in range(len(self.weights)):
            self.weights[i] = cp.clip(self.weights[i], -clip_value, clip_value)

    def plot_error_history(self):
        weight_error_history = cp.array(self.weight_error_history).get()  # Convert to NumPy
        bias_error_history = cp.array(self.bias_error_history).get()  # Convert to NumPy

        def running_mean(data, window_size):
            return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

        num_layers = self.num_layers - 1

        for layer in range(num_layers):
            fig, ax = plt.subplots(figsize=(10, 5))
            layer_weight_errors = weight_error_history[layer::num_layers]
            smoothed_weight_errors = running_mean(layer_weight_errors, window_size=200)
            ax.plot(smoothed_weight_errors, label=f'Weight Update Norms (Layer {layer+1} -> {layer+2})')
            ax.set_title(f'Weight Error Over Epochs (Layer {layer+1} -> {layer+2})')
            ax.set_xlabel('Epochs')
            ax.set_ylabel('Error (Norm)')
            ax.legend()
            plt.tight_layout()
            plt.show()

        for layer in range(num_layers):
            fig, ax = plt.subplots(figsize=(10, 5))
            layer_bias_errors = bias_error_history[layer::num_layers]
            smoothed_bias_errors = running_mean(layer_bias_errors, window_size=200)
            ax.plot(smoothed_bias_errors, label=f'Bias Update Norms (Layer {layer+1} -> {layer+2})')
            ax.set_title(f'Bias Error Over Epochs (Layer {layer+1} -> {layer+2})')
            ax.set_xlabel('Epochs')
            ax.set_ylabel('Error (Norm)')
            ax.legend()
            plt.tight_layout()
            plt.show()

    def train(self, X, y, X_test, y_test, epochs, learning_rate, lower_rate=[2500], batch_size=32):
        num_samples = X.shape[0]

        for epoch in range(epochs):
            if epoch in lower_rate:
                learning_rate = learning_rate / 10

            permutation = cp.random.permutation(num_samples)
            X_shuffled = X[permutation]
            y_shuffled = y[permutation]

            for i in range(0, num_samples, batch_size): 
                X_batch = X_shuffled[i:i + batch_size]
                y_batch = y_shuffled[i:i + batch_size]
                output = self.forward(X_batch)
                self.backward(X_batch, y_batch, output, learning_rate)

            if epoch % 100 == 0:
                # Training metrics
                output = self.forward(X)
                train_mae = cp.mean(cp.abs(y[:, 0] - output[:, 0])).get()
                if self.binary_output:
                    train_auc = roc_auc_score(cp.asnumpy(y[:, 1]), cp.asnumpy(output[:, 1]))
                else:
                    cls_binary_output = (cp.asnumpy(output[:, 1]) >= 6).astype(cp.float32)
                    y_bin = (cp.asnumpy(y[:, 1]) >= 6).astype(cp.float32)
                    train_auc = roc_auc_score(y_bin, cls_binary_output)

                self.training_mae.append(train_mae)
                self.training_auc.append(train_auc)

                # Testing metrics
                predictions = self.predict(X_test)
                test_mae = cp.mean(cp.abs(predictions[:, 0] - y_test[:, 0])).get()
                if self.binary_output:
                    test_auc = roc_auc_score(cp.asnumpy(y_test[:, 1]), cp.asnumpy(predictions[:, 1]))
                else:
                    cls_binary_output = (cp.asnumpy(predictions[:, 1]) >= 6).astype(cp.float32)
                    y_bin = (cp.asnumpy(y_test[:, 1]) >= 6).astype(cp.float32)
                    test_auc = roc_auc_score(y_bin, cls_binary_output)

                self.testing_mae.append(test_mae)
                self.testing_auc.append(test_auc)

                print(f"Epoch {epoch}: Train MAE = {train_mae}, Train AUC = {train_auc}, Test MAE = {test_mae}, Test AUC = {test_auc}")

        self.plot_training_testing_metrics()

    def plot_training_testing_metrics(self):
        epochs = range(0, len(self.training_mae) * 100, 100)
        print("Training MAE:", self.training_mae)  # Debug print
        print("Testing MAE:", self.testing_mae)    # Debug print
        print("Training AUC:", self.training_auc)  # Debug print
        print("Testing AUC:", self.testing_auc)    # Debug print

        plt.figure(figsize=(12, 6))

        # Plot MAE
        plt.subplot(1, 2, 1)
        plt.plot(epochs, self.training_mae, label="Training MAE")
        plt.plot(epochs, self.testing_mae, label="Testing MAE")
        plt.ylim([0, 5])
        plt.title("MAE Over Epochs")
        plt.xlabel("Epochs")
        plt.ylabel("MAE")
        plt.legend()

        # Plot AUC
        plt.subplot(1, 2, 2)
        plt.plot(epochs, self.training_auc, label="Training AUC")
        plt.plot(epochs, self.testing_auc, label="Testing AUC")
        plt.title("AUC Over Epochs")
        plt.xlabel("Epochs")
        plt.ylabel("AUC")
        plt.legend()

        plt.tight_layout()
        plt.show()

    def predict(self, X):
        output = self.forward(X)
        reg_output = output[:, 0]
        if self.binary_output:
            # cls_output = (output[:, 1] >= 0.5).astype(cp.float32)
            cls_output = (output[:, 1]).astype(cp.float32)
        else:
            cls_output = (output[:, 1] >= 6).astype(cp.float32)
        return cp.hstack([reg_output[:, None], cls_output[:, None]])

In [ ]:
historical_hourly_weather_data_path = kagglehub.dataset_download('selfishgene/historical-hourly-weather-data')

city_attributes = pd.read_csv(f"{historical_hourly_weather_data_path}/city_attributes.csv")
humidity = pd.read_csv(f"{historical_hourly_weather_data_path}/humidity.csv")
pressure = pd.read_csv(f"{historical_hourly_weather_data_path}/pressure.csv")
temperature = pd.read_csv(f"{historical_hourly_weather_data_path}/temperature.csv")
weather_description = pd.read_csv(f"{historical_hourly_weather_data_path}/weather_description.csv")
wind_speed = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_speed.csv")
wind_direction = pd.read_csv(f"{historical_hourly_weather_data_path}/wind_direction.csv")

data_frames = []
for city in city_attributes['City']:
    city_data = pd.DataFrame({
        'datetime': pd.to_datetime(humidity['datetime']),
        'humidity': humidity[city],
        'pressure': pressure[city],
        'temperature': temperature[city],
        'weather_description': weather_description[city],
        'wind_speed': wind_speed[city],
        'wind_direction': wind_direction[city],
        'latitude': city_attributes.loc[city_attributes['City'] == city, 'Latitude'].values[0],
        'longitude': city_attributes.loc[city_attributes['City'] == city, 'Longitude'].values[0],
        'city': city
    })
    city_data.set_index('datetime', inplace=True)
    data_frames.append(city_data)

combined_data = pd.concat(data_frames)

combined_data = combined_data.ffill().bfill().interpolate()

aggregated_data = combined_data.groupby(['city']).resample('D').agg({
    'temperature': 'mean',
    'humidity': 'mean',
    'wind_speed': ['max', 'mean'],
    'pressure': 'mean',
    'weather_description': lambda x: x.mode()[0] if not x.mode().empty else np.nan,
    'wind_direction': 'mean',
    'latitude': 'mean',
    'longitude': 'mean'
}).reset_index()

aggregated_data.columns = [
    '_'.join(col).strip('_') if isinstance(col, tuple) else col for col in aggregated_data.columns
]


In [ ]:
one_hot_weather_description = pd.get_dummies(
    aggregated_data['weather_description_<lambda>'],
    prefix='weather_desc',
    drop_first=False
)

aggregated_data = pd.concat([aggregated_data, one_hot_weather_description], axis=1)
aggregated_data.drop(columns=['weather_description_<lambda>'], inplace=True)

In [ ]:
binary_output = True # True better
two_models = True # True = 2.165, Flase = 2.151
standarize_2nd_data = False # False better

In [ ]:
def preprocess_weather_data(data, binary_output=True, window_size=3, two_models=False, standarize_2nd_data=False, dayoffset=1):
    X, y, X_2, y_2 = [], [], [], []
    for i in range(window_size, len(data) - 1):
        X_window = data.iloc[i-window_size:i][[
            'temperature_mean', 'humidity_mean', 'wind_speed_max',
            'wind_speed_mean', 'pressure_mean', 'wind_direction_mean',
            'weather_desc_broken clouds', 'weather_desc_dust',
            'weather_desc_few clouds', 'weather_desc_fog',
            'weather_desc_freezing rain', 'weather_desc_haze',
            'weather_desc_heavy intensity rain', 'weather_desc_light rain',
            'weather_desc_mist', 'weather_desc_moderate rain',
            'weather_desc_overcast clouds', 'weather_desc_scattered clouds',
            'weather_desc_sky is clear', 'weather_desc_smoke', 'weather_desc_snow'
        ]].values
        y_target = data.iloc[i + dayoffset][['temperature_mean', 'wind_speed_max']].values


        if binary_output:
            # encode wind speed > 6 as binary
            y_target[1] = 1 if y_target[1] >= 6 else 0
            if two_models:
                y_target2[1] = 1 if y_target2[1] >= 6 else 0
        
        X.append(X_window)
        y.append(y_target)
        if two_models:
            X_2.append(y_target)
            y_2.append(y_target2)

    X = np.array(X)
    y = np.array(y)
    X_2 = np.array(X_2)
    y_2 = np.array(y_2)

    # split into train/test (0.7 or 0.8)
    train_size = int(0.8 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    continuous_indices = [0, 1, 2, 3, 4, 5]

    X_train_continuous = X_train[:, :, continuous_indices].astype(float) 
    X_test_continuous = X_test[:, :, continuous_indices].astype(float)

    X_mean = X_train_continuous.mean(axis=(0, 1), keepdims=True)
    X_std = X_train_continuous.std(axis=(0, 1), keepdims=True)

    X_train[:, :, continuous_indices] = (X_train_continuous - X_mean) / (X_std + 1e-9)
    X_test[:, :, continuous_indices] = (X_test_continuous - X_mean) / (X_std + 1e-9)


    return X_train, X_test, y_train, y_test


In [ ]:
X_train, X_test, y_train, y_test = preprocess_weather_data(aggregated_data, window_size=3, dayoffset=0)

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=1000, learning_rate=0.001,rate = [500], batch_size=256):
    X_train_cp = cp.array(X_train.reshape(X_train.shape[0], -1), dtype=cp.float32)
    y_train_cp = cp.array(y_train, dtype=cp.float32)
    X_test_cp = cp.array(X_test.reshape(X_test.shape[0], -1), dtype=cp.float32)
    y_test_cp = cp.array(y_test, dtype=cp.float32)

    model.train(X=X_train_cp, y=y_train_cp, X_test=X_test_cp, y_test=y_test_cp, epochs=epochs, learning_rate=learning_rate ,lower_rate = rate,batch_size=batch_size )

    predictions = model.predict(X_test_cp)
    
    mae = cp.mean(cp.abs(predictions[:, 0] - y_test_cp[:, 0]))

    from sklearn.metrics import roc_auc_score
    auc = roc_auc_score(cp.asnumpy(y_test_cp[:, 1]), cp.asnumpy(predictions[:, 1]))

    print(f"Test Regression MAE: {mae}")
    print(f"Test Classification AUC: {auc}")

    return mae, auc

In [ ]:
#predykcja na dzisiaj
# X_train, X_test, y_train, y_test = preprocess_weather_data(aggregated_data, window_size=3)

print(X_train.shape[1])
layers = [X_train.shape[1] * X_train.shape[2],64, 32, 2]
activations = ["sigmoid", "relu"]
model_today = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model_today, X_train, y_train, X_test, y_test, epochs=200, learning_rate=0.0001, rate = [500, 750], batch_size=128)

In [ ]:
def preprocess_weather_data_with_today_prediction(model = model_today):
    X_train2, X_test2, y_train2, y_test2 = preprocess_weather_data(aggregated_data, window_size=3, dayoffset=1)
    X_train_cp = cp.array(X_train.reshape(X_train.shape[0], -1), dtype=cp.float32)
    y_train_cp = cp.array(y_train, dtype=cp.float32)
    X_test_cp = cp.array(X_test.reshape(X_test.shape[0], -1), dtype=cp.float32)
    y_test_cp = cp.array(y_test, dtype=cp.float32)
    expanded_X_train = []
    for sample in X_train_cp:
        # Predict for each individual sample
        prediction = model.predict(sample.reshape(1, -1))  # Reshape to 2D array (1, features)
        expanded_sample = cp.hstack([sample, prediction.flatten()])  # Add prediction to sample
        expanded_X_train.append(expanded_sample)

    expanded_X_test = []
    for sample in X_test_cp:
        # Predict for each individual sample
        prediction = model.predict(sample.reshape(1, -1))  # Reshape to 2D array (1, features)
        expanded_sample = cp.hstack([sample, prediction.flatten()])  # Add prediction to sample
        expanded_X_test.append(expanded_sample)

    # Convert the expanded lists back into numpy arrays
    expanded_X_train = cp.array(expanded_X_train)
    expanded_X_test = cp.array(expanded_X_test)
    return expanded_X_train, expanded_X_test, y_train_cp, y_test_cp

In [ ]:
X_train2, X_test2, y_train2, y_test2 = preprocess_weather_data_with_today_prediction()
# X_train = X_train[:, 6:]
# X_test = X_test[:, 6:]
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

In [ ]:
#predykcja na jutro z predykcją na dzisiaj

layers = [X_train.shape[1], 64,32, 2]
activations = ["sigmoid","linear",]
model2 = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model2, X_train2, y_train2, X_test2, y_test2, epochs=200, learning_rate=0.00001, rate = [],batch_size=128)

In [ ]:
#predykcja na jutro z predykcją na dzisiaj

layers = [X_train.shape[1], 64,32, 2]
activations = ["sigmoid","linear",]
model2 = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model2, X_train2, y_train2, X_test2, y_test2, epochs=500, learning_rate=0.0001, rate = [],batch_size=256)

In [ ]:
#predykcja na jutro z predykcją na dzisiaj

layers = [X_train.shape[1], 64,32, 2]
activations = ["sigmoid","relu",]
model2 = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model2, X_train2, y_train2, X_test2, y_test2, epochs=500, learning_rate=0.0001, rate = [],batch_size=256)

In [ ]:
#predykcja na jutro z predykcją na dzisiaj

layers = [X_train.shape[1], 128,64, 2]
activations = ["sigmoid","linear",]
model2 = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model2, X_train2, y_train2, X_test2, y_test2, epochs=500, learning_rate=0.0001, rate = [],batch_size=256)

In [ ]:
#predykcja na jutro z predykcją na dzisiaj

layers = [X_train.shape[1], 256,128, 2]
activations = ["sigmoid","linear",]
model2 = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model2, X_train2, y_train2, X_test2, y_test2, epochs=400, learning_rate=0.0001, rate = [],batch_size=256)

In [ ]:
#predykcja na jutro z predykcją na dzisiaj

layers = [X_train.shape[1], 512,512, 2]
activations = ["sigmoid","linear",]
model2 = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model2, X_train2, y_train2, X_test2, y_test2, epochs=400, learning_rate=0.00001, rate = [],batch_size=256)

In [ ]:
#predykcja na jutro z predykcją na dzisiaj

layers = [X_train.shape[1], 512,512, 2]
activations = ["sigmoid","linear",]
model2 = WeatherPredictionNetwork(layers, activations, seed=42, binary_output=True)

train_and_evaluate(model2, X_train2, y_train2, X_test2, y_test2, epochs=1000, learning_rate=0.00001, rate = [500],batch_size=256)

In [ ]:
# from itertools import product
# import cupy as cp
# from sklearn.metrics import roc_auc_score

# def hyperparameter_optimization(train_and_evaluate, X_train, y_train, X_test, y_test, binary_output):
#     # Zakres hiperparametrów do optymalizacji
#     layer_options = [
#         [X_train.shape[1] * X_train.shape[2], 256, 128, 2],
#         [X_train.shape[1] * X_train.shape[2], 512, 512, 2],
#         [X_train.shape[1] * X_train.shape[2], 1024, 512, 2]
#     ]
#     activation_options = [
#         ["sigmoid", "relu"],
#         ["relu", "relu"],
#         ["tanh", "relu"]
#     ]
#     learning_rate_options = [0.005, 0.0001, 0.00001]
#     batch_size_options = [64, 128, 256]
#     epochs_options = [500, 1000]

#     # Lista wszystkich kombinacji hiperparametrów
#     search_space = list(product(layer_options, activation_options, learning_rate_options, batch_size_options, epochs_options))

#     best_mae = float("inf")
#     best_auc = 0
#     best_params = None

#     # Iteracja po wszystkich kombinacjach hiperparametrów
#     for layers, activations, learning_rate, batch_size, epochs in search_space:
#         print(f"Testing layers={layers}, activations={activations}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
        
#         # Tworzenie modelu
#         model = WeatherPredictionNetwork(layers, activations, binary_output=binary_output, seed=42)

#         # Trening i ewaluacja
#         mae, auc = train_and_evaluate(
#             model, X_train, y_train, X_test, y_test,
#             epochs=epochs, learning_rate=learning_rate, rate=[500, 750, 900],
#             batch_size=batch_size, binary_output=binary_output
#         )

#         # Aktualizacja najlepszych hiperparametrów
#         if mae < best_mae or (mae == best_mae and auc > best_auc):
#             best_mae = mae
#             best_auc = auc
#             best_params = (layers, activations, learning_rate, batch_size, epochs)

#         print(f"Finished testing with MAE: {mae}, AUC: {auc}\n")

#     print(f"Best params: layers={best_params[0]}, activations={best_params[1]}, learning_rate={best_params[2]}, batch_size={best_params[3]}, epochs={best_params[4]}")
#     print(f"Best MAE: {best_mae}, Best AUC: {best_auc}")

#     return best_params, best_mae, best_auc


In [ ]:
# best_params, best_mae, best_auc = hyperparameter_optimization(
#     train_and_evaluate,
#     X_train, y_train,
#     X_test, y_test,
#     binary_output=binary_output
# )

In [ ]:
# # %pip install scikit-optimize
# from skopt import gp_minimize
# from skopt.space import Integer, Real, Categorical
# from skopt.utils import use_named_args
# import numpy as np

# def bayesian_optimization(train_and_evaluate, X_train, y_train, X_test, y_test, binary_output):
#     # Przestrzeń wyszukiwania dla hiperparametrów
#     space = [
#         Categorical(["config_0", "config_1", "config_2", "config_3"], name="layers"),
#         Categorical(["sigmoid_relu", "sigmoid_linear"], name="activations"),  # Zakodowane krotki jako stringi
#         Real(1e-4, 1e-3, prior="log-uniform", name="learning_rate"),
#         Categorical([64, 128, 256], name="batch_size"),
#         Categorical([50, 200, 500], name="epochs")
#     ]

#     # Mapowanie z ciągów znaków na rzeczywiste konfiguracje warstw
#     layers_mapping = {
#         "config_0": (X_train.shape[1] * X_train.shape[2], 64, 32, 2),
#         "config_1": (X_train.shape[1] * X_train.shape[2], 128, 64, 2),
#         "config_2": (X_train.shape[1] * X_train.shape[2], 256, 128, 2),
#         "config_3": (X_train.shape[1] * X_train.shape[2], 512, 256, 2)
#     }

#     # Mapowanie string `activations` na rzeczywistą krotkę
#     activations_mapping = {
#         "sigmoid_relu": ("sigmoid", "relu"),
#         "sigmoid_linear": ("sigmoid", "linear")
#     }

#     @use_named_args(space)
#     def objective(layers, activations, learning_rate, batch_size, epochs):
#         # Mapowanie string `layers` na rzeczywistą konfigurację
#         mapped_layers = layers_mapping[layers]
#         # Mapowanie string `activations` na rzeczywistą krotkę
#         mapped_activations = activations_mapping[activations]
#         print(f"Testing: layers={mapped_layers}, activations={mapped_activations}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")

#         # Tworzenie modelu
#         model = WeatherPredictionNetwork(list(mapped_layers), list(mapped_activations), binary_output=binary_output, seed=42)

#         # Trening i ewaluacja
#         mae, auc = train_and_evaluate(
#             model, X_train, y_train, X_test, y_test,
#             epochs=epochs, learning_rate=learning_rate, rate=[500, 750, 900],
#             batch_size=batch_size, binary_output=binary_output
#         )

#         print(f"MAE: {mae}, AUC: {auc}")

#         # Zwróć MAE jako wartość skalarną
#         return float(mae)

#     # Optymalizacja przy użyciu Bayesian Optimization
#     result = gp_minimize(
#         func=objective,
#         dimensions=space,
#         n_calls=10,  # Liczba iteracji optymalizacji (tutaj 20)
#         n_initial_points=2,
#         random_state=30,
#         verbose=True
#     )

#     # Najlepsze hiperparametry
#     best_params = result.x
#     print(f"Best parameters: layers={best_params[0]}, activations={best_params[1]}, learning_rate={best_params[2]}, batch_size={best_params[3]}, epochs={best_params[4]}")
#     print(f"Best MAE: {result.fun}")

#     return best_params, result.fun


In [ ]:
# best_params, best_mae = bayesian_optimization(
#     train_and_evaluate,
#     X_train, y_train,
#     X_test, y_test,
#     binary_output=binary_output
# )

In [ ]:
# def compute_majority_class_percentage(y, binary_output=True):
#     if binary_output:
#         binary_labels = y[:, 1]
#     else:
#         binary_labels = y[:, 1] >=6
    
#     unique, counts = np.unique(binary_labels, return_counts=True)
#     class_counts = dict(zip(unique, counts))
    
#     majority_class_count = max(class_counts.values())
#     total_samples = len(binary_labels)
#     majority_class_percentage = (majority_class_count / total_samples) * 100
    
#     return majority_class_percentage, class_counts

# majority_percentage_train, train_class_counts = compute_majority_class_percentage(y_train, binary_output=binary_output)
# majority_percentage_test, test_class_counts = compute_majority_class_percentage(y_test, binary_output=binary_output)

# print(f"Majority class percentage in training data: {majority_percentage_train:.2f}%")
# print(f"Class distribution in training data: {train_class_counts}")
# print(f"Majority class percentage in test data: {majority_percentage_test:.2f}%")
# print(f"Class distribution in test data: {test_class_counts}")
